# Code Efficiency

Python is a very versatile programming language which allows for code to be run in real-time and scripts to be changed on the fly. Furthermore, Python has many quality of life aspects such as adaptive memory allocation that make programming easy. However, these aspects come at a cost. This cost is computational speed. In general, a Python program executing some code will be much slower than compiled programming languages such as C. 

In order to have your code perform efficiently, you will need to understand which parts of your code are slow and what options you have to speed things up.

### What you will learn

In this notebook we will cover the following topics:
* comparing different loops
* redundant operations
* built-in Python code, falling back to fast compiled code, and special libraries
* keeping data in memory vs on-the-fly computation
* comprehensions and the fuctions `lambda`, `map`, and `filter`

It should be noted that there are two idependent notions of code efficiency discussed here. On one hand, we will be looking at how fast code will be executed and thus more efficient code will perform a computation faster. On the other hand, we will also discuss code efficiency in terms of compactness of code, that is, more efficient code is written with fewer lines. Although the two concepts are different, they are related. More compact code tends to execute faster and faster code tends to be more compact. Thus, although we will move away from looking at execution speed in the second half of this notebook and focus more on compact code, using these techniques will likely improve your code performance. For this very reason, using these compact coding techniques are industry standard. 

*&#169; Tobias Hartung, University of Bath 2021-2025. This problem sheet is copyright of Tobias Hartung, University of Bath. It is provided exclusively for educational purposes at the University and is to be downloaded or copied for your private study only. Further distribution, e.g. by upload to external repositories, is prohibited.*

## Loops

Let us consider the example of summing the numbers from 0 to N-1 for some large number N. A straightforward approach to such a task would be to use loops. The primary choice here is between a while loop and a for loop.

In [ ]:
def while_sum(N=10**6):
    i = 0
    s = 0
    while i<N:
        s += i
        i += 1
    return s

def for_sum(N=10**6):
    s = 0
    for i in range(N):
        s += i
    return s



In both cases we do effectively the same. We initialize the sum as `s = 0` and then loop over all numbers i from 0 to N-1 adding to the sum. 

We can test which of these functions is faster by using the `timeit` function from the timeit library.This function is the same as the `%timeit` directive you have encountered in Semester 1, but it provides more customizability which we need here. Using just `%timeit` would mean this notebook would need years to execute.

In [ ]:
import timeit
print('while loop\t\t',timeit.timeit(while_sum, number=1))
print('for loop\t\t',timeit.timeit(for_sum, number=1))



Although these two functions do essentially the same, we can see that the while loop took almost twice as long as the for loop. Why is that? 

Python is written in the programming language C, which is a compiled language and much faster Python itself. If we look at the difference in our two functions, the while loop increments the index variable i and checks whether i<N using Python, whereas the for loop passes both of these steps on to the `range(N)` object which is written in C. Effectively, the while loop does three slow operations in each loop (`i<N`, `s+=i`, `i+=1`) whereas the for loop uses one slow (`s+=i`) and two fast operations (`i<n` and `i+=1`). 

We can confirm this, by artificially adding the increment and the check `i<N` into the for loop. To this end, we introduce the `pass` statement. This statement literally tells Python to do nothing. In real code development it is used as a placeholder that allows you to execute code although parts have not yet been written. Here we use it as a line `if j<N: pass` which forces the if-statement to execute but then do nothing at all. So, the code below formally does the same if-check that the while loop does and increments the a counter variable just like the while loop does, but nothing else is executed. Hence, we can see the impact of the two additional Python statements that are present in the while loop and compare it to the performance of a pure for loop. 

In [ ]:
def slow_for_sum(N=10**6):
    s = 0
    j = 0
    for i in range(N):
        if j<N: pass
        s += i
        j += 1
    return s

print('slow for loop\t\t',timeit.timeit(slow_for_sum, number=1))



This slow_for_sum should be even slower than the while_sum because now the test `i<N` and the increment `i+=1` is done twice; once in Python, once in C. This also highlights our first important observation.

#### Redundant operations can slow down your code dramatically

Now the next question to ask is: Is there anything we can do to go faster than a for loop?

Since we are summing numbers, we may use the built-in function sum.

In [ ]:
def built_in_sum(N=10**6):
    return sum(range(N))

print('built in sum\t\t',timeit.timeit(built_in_sum, number=1))

Now this sum function again uses a for loop but this time it is the for loop written in C and not in Python. This is why this function is a lot faster and it brings us to our second observation.

#### Use built-in functions over functions you write yourself. 

Of course, writing code yourself and ensuring it works as intended is very good for code development and debugging, but once you are moving on to production code, replacing your hand-written function with built-ins where possible can really improve code performance. 

Yet, we can do even better than that. Python is a general purpose programming language. It is a jack of all trades. Thus, if you are looking for special interest applications, it is often a good idea to use special libraries. For example, Numpy is a library written to do mathematics quickly. As you have already encountered Numpy quite extensively in semester 1, we will not discuss any explicit functionality of Numpy here, but rather aim to understand what Numpy does "under the hood" that makes it so much better than pure Python.

So, what happens if we replace the sum and range objects in the last function with their Numpy versions?

In [ ]:
import numpy

def numpy_sum(N=10**6):
    return numpy.sum(numpy.arange(N))

print('Numpy sum\t\t',timeit.timeit(numpy_sum, number=1))

Numpy is primarily written in C, so why is it so much faster than the Python built-in version? There are two important aspects to why this one is so much faster. 

1. Python uses arbitrary precision numbers. If a number becomes too large for the currently allocated memory, Python will give it more space. This means that Python is checking whether or not the numbers still fit into memory. This operation takes time. Numpy, on the other hand, has fixed memory allocations. It therefore does not need to check whether there is sufficient space and thus saves time. However, if you do run out of memory for your numbers, you will get overflow errors and other problems you will have to deal with. 

2. The Python range object is a so-called generator object. It generates one number at a time and only once the for loop inside the sum function requires the next number, the next number will be computed. The numpy.arange object computes all elements in the range at the beginning. It is therefore effectively a list with all the numbers already available at the time the `numpy.sum` function starts looping. This means that `numpy.sum(numpy.arange(N))` is one big C call, whereas `sum(range(N))` still contains Python overhead in the generator `range(N)`. Conversely, because `numpy.arange(N)` creates the entire array in memory, we have just created 100 million numbers in memory. If N were larger, we could run out of memory, i.e., crash the computer. The Python version `sum(range(N))` will run no matter how large N is, provided that N^2 fits into memory. 

In many applications, memory constraints are not as critical as speed. As such, it is worth giving up the memory and ensuring yourself that enough memory is available, to let the sum happen in C and get your results faster. In any case, this leads us to our next observation.

#### Use purpose-built libraries like Numpy, Scipy, Pandas, Sympy, ... for their intended applications over pure Python.

Numpy is a core library for scientific computing. Many other scientific libraries either mirror the Numpy structure or explicitly build on top of Numpy. In particular, Numpy provides a high-performance multi-dimensional array object and the tools for working with them. 

As we have alluded to earlier, Numpy has fixed data types that allow for faster operations. You can set the datatype using the dtype argument and you can also let a Numpy array tell you its datatype using the `dtype` attribute. 

In [ ]:
import numpy as np

x = np.array([1, 2])   # Let numpy choose the datatype
print(x.dtype)         # Prints "int64"

x = np.array([1.0, 2.0])   # Let numpy choose the datatype
print(x.dtype)             # Prints "float64"

x = np.array([1, 2], dtype=np.complex128)   # Force a particular datatype
print(x.dtype)                              
print(x)

This is very important for code efficiency as knowing the datatype saves a lot of overhead Python needs to do in checking datatypes. Therefore, if performance is important, numpy is the best starting point for numerical mathematics. 

While Numpy provides basic arrays and their functionality, SciPy expands on Numpy with many useful tools for scientific and engineering applications. These include image manipulation, data fitting, generation of numerical integration data, and even interoperability with MATLAB files. Similarly, if you are working with large data sets such as in machine learning, then Pandas is a very suitable starting point.

As a sidenote, the Numpy implementation of this sum reveals a neat order of magintude estimation of hardware specifications. `numpy_sum` does 10^6 additions in its execution. Adding two numbers is a basic operation for the CPU and executes in a single CPU cycle. So, if 10^6 operations require about 10^(-3) seconds, that means that each addition takes about 10^(-9) seconds. From this, we can compute how often my CPU cycles every second; namely 10^9 cycles per second. Note that 1 cycle per second is 1Hz, 1000 cycles/second is 1kHz, 10^6 cycles/second is 1MHz, and 10^9 cycles/second is 1GHz. So my laptop has a CPU that has a clock frequency a few GHz. In fact, most CPUs today run a clock between 2GHz and 5GHz, so this is a fairly decent order of magnitude estimate of how fast my computer can compute.

##### But can we do even better?

Now you may wonder whether there is anything we can do to beat Numpy. Frankly, you will be hard pressed to find a better implementation of `numpy.sum(numpy.arange(N))` even in C directly. Numpy does a very good job at implementing basic mathematics. If we were to implement `numpy.sum(numpy.arange(N))` directly in machine code, you could get about a factor 10 on the performance, but there are very few applications in which this effort is justifiable. Alternatively, you could try to split the sum into parts and process them in parallel before eventually summing up the individual results. In that case you would get an improvement factor of roughly the number of CPUs you have available. Thus, realistically, `numpy.sum(numpy.arange(N))` is about as optimized as it is ever going to get.

Therefore, the only feasible way of making things faster is to be smart about it. After all, there is a point to mathematics and in this case, we know that the sum of numbers from 0 to N-1 is given by `N*(N-1)/2`. So, what if we implement this formula?

In [ ]:
def smart_sum(N=10**6):
    return N*(N-1)//2

print('smart sum\t\t',timeit.timeit(smart_sum, number=1))

Numpy is good, but using mathematics knowledge, you cannot even compete with that. 

#### Use your mathematics education to really speed up the code!

So, what is the fastest way of doing a lot of operations in Python? Not doing a lot of operations in Python but using mathematics to simplify what needs to be done. If you can no longer simplify your computation, use purpose-build libraries. After that, your next best option are built-in functions in Python. Only if all of these options are exhausted, should you try to loop by hand. In this case, use for loops over while loops whenever possible. 

## Comprehensions, lambda, map, and filter

### Comprehensions

Loops are often hard to understand, especially if they are to define objects. You can often make your code shorter (thus faster) and clearer by using comprehensions. The gerenal form of a comprehension is

`expression for item in iterable if condition == True`

although the `if condition == True` can be omitted. 

For example, consider a list of fruits and you want to create a new list of fruits you like, based on fruits you don't like.


In [ ]:
fruits = ['apple','banana','cherry','guava','kiwi','mango','pear']
disliked_fruits = ['apple','guava']

liked_fruits = [f for f in fruits if f not in disliked_fruits]
print(liked_fruits)

# for comparison: the same code but without using comprehensions
liked_fruits_2 = [] # define empty list
for fruit in fruits: # check all fruits
    if fruit not in disliked_fruits: # check that it is not disliked
        liked_fruits_2.append(fruit) # if so, add it to liked fruits
print(liked_fruits_2)

This can also allow you to replace objects in more complicated settings. 

In [ ]:
replaced_fruits = [f if f !='banana' else 'orange' for f in fruits]
print(replaced_fruits)

# for comparison: the same code but without using comprehensions
replaced_fruits_2 = [] # define empty list
for fruit in fruits: # check all fruits
    if fruit != 'banana': # check that it is not a banana
        replaced_fruits_2.append(fruit) # if so, add it to list of replaced fruits
    else: # but if it is a banana
        replaced_fruits_2.append('orange') # add an orange instead
print(replaced_fruits_2)

The example above, where the comprehension is inside a list, is called a list comprehension. But there are many other options. You can also have set, dictionary, and even generator comprehensions.

In [ ]:
list_comp = [i**2 for i in range(10)]   # list comprehension computing squares up to 9^2
dic_comp = {i:i**2 for i in range(10)}  # dictionary comprehension computing squares up to 9^2
set_comp = {i%3 for i in range(10)}     # set comprehension computing number mod 3 up to 9 mod 3
gen_comp = (2*x+5 for x in range(10))   # generator comprehension computing numbers 2x+5 up to x=9

print('list comprehension: ', list_comp)
print('dictionary comprehension:',dic_comp)
print('set comprehension:',set_comp)
print('generator comprehension:',gen_comp)
for i in gen_comp:
    print(i)

### Lambda

When it comes to mathematics and comprehensions, we often want to define functions quickly or anonymously. Lambda functions can make this happen for us. The lambda notation consists of the keyword lambda, a list of variables, and the body of the function.

In [ ]:
square = lambda x: x**2
add = lambda x,y:x+y
print(square(2))
print(add(5,7))

# alternative without using lambda
def square_2(x):
    return x**2
def add_2(x,y):
    return x+y

An anonymous function is a function without a name. You could define one as

In [ ]:
lambda x,y: x*y

although there is not much of a point by itself. Instead, anonymous functions are often used if invoked immediately and never again or if passed on to another function.

In [ ]:
print((lambda x: x+2)(4)) # immediately invoking the function

# or defining an evaluation function
def eval(func,x):
    print(func(x))

# example of evaluating an anonymous function directly in the eval call
eval(
    lambda x:x-2, # passing a function into the eval function
    10
) 

# alternative without using lambda
def function_I_only_need_once(x): # defining a named function
    return x-2

eval( 
    function_I_only_need_once, # passing a named function into eval function
    10 
)

As you can see, using anonymous function not only declutters the code, but more importantly, it means you do not need to define another function which stays inside your namespace and may cause issues because you forgot it existed. The anonymous function is gone as soon as `eval` finishes executing. In large, memory intensive programs, this can also help manage memory usage during execution.

Eventually, the key to why people use lambda functions is that it allows you to write very compact and readable code; especially if you combine lambda functions with comprehensions of functions like `sum`, `map`, and `filter` (below).

### map and filter

Lambda functions are most commonly used with built-in functions such as `sum`, `map`, or `filter`. 

`filter(function, iterable)` is equivalent to the generator comprehension `(item for item in iterable if function(item))`. Thus, you want a function to give either `True` or `False` and `filter` generates all elements of the iterable for which the function is True. For example, if we wish to find all animals in a list whose name has an "o" in it, then we could write.

In [ ]:
list(filter(lambda x: 'o' in x, ['cat', 'dog', 'cow']))

where we used the anonymous function `lambda x: 'o' in x` which returns `True` if and only if the element `x` of the the list `['cat', 'dog', 'cow']` has an `o` in it. Also, note that filter returns `filter` object which is a generator. Hence, for us to read the filtered list, we use `list()` to cast the generator into a list object. 

The other main application for lambda functions is the `map` function. `map(function,iterable)` produces a generator object that applies the function to each element of the iterable. Again, to be able to read the output, we will cast it into a list. 

In [ ]:
print(list(map(lambda x: x**2, range(5))))
print([x**2 for x in range(5)])

As you can see, the combination of lambda, map, and other built-in functions allows you to save on many loops and thus computation time. 


# Check your understanding

##### Question 1
Which of the answers is equivalent to
```
def squares(n):
    for j in range(n):
        yield j**2
```

```
A squares = (j**2 for j in range(n))
B squares = [j**2 for j in range(n)]
C squares = {j**2 for j in range(n)}
D squares = map(lambda j: j**2, range(n))
```

##### Question 2
Order the following implementations by expected speed of execution for large `N` (fast to slow). 
```
A 
numpy.sum(numpy.arange(N))

B
s = 0
for j in range(N):
    s += j
    
C 
sum(range(N))

D 
s = 0
j = 0
while j<N:
    s += j
    j += 1

E 
sum(numpy.arange(N))
```


```























```

# Answers
Q1: A, D 
Q2: A, E, C, B, D